# Imports

In [1]:
from keras import backend as K
import time
import matplotlib.pyplot as plt
import numpy as np_utils
%matplotlib inline

from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, DepthwiseConv2D, Conv2D, SeparableConv2D, MaxPooling1D
from keras.layers import Input, concatenate
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Nadam, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l2
%config InlineBackend.figure_format='retina'
from keras_contrib.callbacks import CyclicLR
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from data_science_utils.vision.keras import *
from time import time
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import missingno as msno
import re
from joblib import Parallel, delayed
from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils import plots as plot_utils
from data_science_utils.dataframe import column as column_utils
from data_science_utils import misc as misc
from data_science_utils import preprocessing as pp_utils
from data_science_utils import nlp as nlp_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from data_science_utils.dataframe import get_specific_cols

import numpy as np
import more_itertools
from more_itertools import flatten
import ast
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw to /home/ec2-user/nltk_data...
[nltk_data]   Package omw is already up-to-date!


In [2]:
df_train = pd.read_csv("asin_classification/train_file")
df_test = pd.read_csv("asin_classification/test_file")

In [4]:
df_train.columns

Index(['feature', 'target'], dtype='object')

In [5]:
df_train.head()
df_test.head()

,feature,target
0,EASY4BUY® SAI Baba Waterproof Car Bike Decal S...,26307599
1,adidas Originals Men's Gazelle Vappnk White an...,30904020
2,SG Musical 36cm Six Holes Finest Indian Bansur...,26706100
3,Spykar Mens Blue Skinny Fit Low Rise Jeans (31),19301050
4,TOMMY HILFIGER Men's Printed Regular Fit Casua...,19301621


,ID,feature
0,320bcdc104345482cb5e6b2f96f8b404,ARTBOX Kids Printed Diary in a Beautiful Pack ...
1,9bfc6e30373362f1b814df3c4378ff0c,L'Oreal Paris Revitalift Eye Night Cream 15ml
2,e169087ecdf5991987792e11eaa5fda6,Agro Fresh Diamond Sugar 100g
3,f8c575910b94818d484cd8e9c6a62b94,Raymond Home Exclusive 104 TC Cotton Double Be...
4,f3501cc33c4620754960d9211fbfded9,DAMDAM Women's Handbag (BLACK-EWDD-0001)


# Preprocessing Data

In [6]:
import sys
import os
sys.path.append(os.getcwd())
from importlib import reload
import lib
reload(lib)
from lib import *

<module 'lib' from '/home/ec2-user/SageMaker/ML_hackathon_2019/lib.py'>

In [4]:
df_train = preprocess_for_word_cnn(df_train,text_columns=['feature'],jobs=32)
gl_le_train,gl_le_transform, le = get_text_le("text")
_ = gl_le_train(df_train)
df_train['text_encoded'] = gl_le_transform(df_train)




Total Words before Min frequency filtering 332242
Total Words after Min frequency filtering 87552


In [5]:
df_test = preprocess_for_word_cnn(df_test,text_columns=['feature'],jobs=32)
df_test['text_encoded'] = gl_le_transform(df_test)

In [9]:
df_test.head()

,ID,feature,text,text_encoded
0,320bcdc104345482cb5e6b2f96f8b404,ARTBOX Kids Printed Diary in a Beautiful Pack ...,"[artbox, kid, print, diary, beautiful, pack, p...","[13263, 51, 13, 765, 412, 16, 244, 88, 103]"
1,9bfc6e30373362f1b814df3c4378ff0c,L'Oreal Paris Revitalift Eye Night Cream 15ml,"[oreal, paris, revitalift, eye, night, cream, ...","[1942, 1317, 19849, 386, 295, 203, 8]"
2,e169087ecdf5991987792e11eaa5fda6,Agro Fresh Diamond Sugar 100g,"[agro, fresh, diamond, sugar, _NUM6_]","[2782, 553, 381, 982, 11]"
3,f8c575910b94818d484cd8e9c6a62b94,Raymond Home Exclusive 104 TC Cotton Double Be...,"[raymond, home, exclusive, _NUM6_, cotton, dou...","[1222, 71, 827, 11, 12, 117, 298, 4, 166, 10, 22]"
4,f3501cc33c4620754960d9211fbfded9,DAMDAM Women's Handbag (BLACK-EWDD-0001),"[damdam, woman, handbag, black, ewdd, _NUM1_]","[38728, 5, 476, 9, 1, 4]"


In [10]:
df_train.head()

,feature,target,text,text_encoded
0,EASY4BUY® SAI Baba Waterproof Car Bike Decal S...,26307599,"[easy, _NUM2_, buy, sai, baba, waterproof, car...","[475, 2, 855, 1514, 2659, 235, 60, 225, 537, 1..."
1,adidas Originals Men's Gazelle Vappnk White an...,30904020,"[adidas, original, men, gazelle, vappnk, white...","[1007, 329, 7, 22746, 1, 22, 1, 36, 99, 3, 30, 6]"
2,SG Musical 36cm Six Holes Finest Indian Bansur...,26706100,"[musical, _NUM5_, _NUM2_, hole, fine, indian, ...","[974, 6, 2, 1205, 806, 223, 6245, 7499, 9, 108..."
3,Spykar Mens Blue Skinny Fit Low Rise Jeans (31),19301050,"[spykar, men, blue, skinny, fit, low, rise, je...","[2333, 7, 17, 528, 19, 606, 190, 97, 6]"
4,TOMMY HILFIGER Men's Printed Regular Fit Casua...,19301621,"[tommy, hilfiger, men, print, regular, fit, ca...","[787, 815, 7, 13, 37, 19, 31, 14, 3, 1964, 11,..."


In [11]:
df_train.target.nunique()

1316

In [12]:
# Intermediate save
df_test[["ID","feature","text","text_encoded"]].to_csv("asin_classification/test.csv",index=False)
df_train[["feature","text","text_encoded","target"]].to_csv("asin_classification/train.csv",index=False)



In [15]:
# What is the max word length of an asin?

max_len = 0
for c in df_train.text:
    max_len = max(max_len,len(c))
print(max_len)

df_train.text.apply(len).mean()
df_train.text.apply(len).max()

np.percentile(df_train.text.apply(len), [1,5,95,99])

558


43.8424498209171

558

array([  4.,   9., 108., 158.])

In [16]:
# What is the max character length of an asin?

max_len = 0
for c in df_train.char:
    max_len = max(max_len,len(c))
print(max_len)

df_train.char.apply(len).mean()

np.percentile(df_train.char.apply(len), [1,5,95,99])

4186


355.25368168497965

array([  41.,   69.,  955., 1423.])

4186

In [ ]:
start = time()
glove = api.load("glove-twitter-25") 
print("total = ",(time()-start))

ptr_glove = PreTrainedEmbeddingsTransformer(glove,size=25)
ptr_glove.fit()


In [18]:
ptr_glove.transform(df_train['text'].head().values)

Fasttext Transforms start at: 2019-06-12 20:48:33.769052
Number of Unique Test Tokens for Fasttext transform 105


/home/ec2-user/SageMaker/ML_hackathon_2019/lib.py:230: RuntimeWarning: invalid value encountered in true_divide
  token2vec = {k: self.model.wv[k] if k in self.model.wv else empty for k in uniq_tokens}


Fasttext Transforms done at: 2019-06-12 20:48:33.798613


array([list([array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.]), array([-0.34577322, -0.24588524,  0.05102374, -0.10197101,  0.08651397,
        0.07678932, -0.1665935 , -0.3043444 ,  0.32156006, -0.18454368,
        0.17527086,  0.09763581, -0.34966585, -0.08168061, -0.13189802,
        0.0683877 ,  0.09058966,  0.13328592,  0.16063175,  0.39769828,
       -0.30992848,  0.00171429,  0.18304455,  0.02893994, -0.06448579],
      dtype=float32), array([-0.3268838 , -0.13950849, -0.07745891, -0.19274539,  0.14657307,
       -0.13236557, -0.09916801, -0.3344287 , -0.03204315, -0.14627622,
        0.25205323, -0.01479604, -0.4430464 , -0.06483459, -0.2673904 ,
       -0.09216734,  0.11876208, -0.17181139,  0.07366584,  0.25502172,
       -0.27456424, -0.23463401, -0.1821145 ,  0.0150461 , -0.15898505],
      dtype=float32), array([ 0.04465171, -0.1731172 , -0.01628751, -0.11452806,  0.32375216,
       -0.02755725,  0.10969554, 

In [20]:

df_test[["ID","GL","text","text_encoded","char","char_encoded"]].to_csv("price_prediction/test.csv",index=False)
df_train[["ID","GL","text","text_encoded","char","char_encoded","PRICE"]].to_csv("price_prediction/train.csv",index=False)


In [19]:
df_test['glove_encoded'] = ptr_glove.transform(df_test['text'].values)
df_train['glove_encoded'] = ptr_glove.transform(df_train['text'].values)


df_test[["ID","GL","text","text_encoded","char","char_encoded","glove_encoded"]].to_csv("price_prediction/test.csv",index=False)
df_train[["ID","GL","text","text_encoded","char","char_encoded","glove_encoded","PRICE"]].to_csv("price_prediction/train.csv",index=False)


Fasttext Transforms start at: 2019-06-12 20:49:31.612184
Number of Unique Test Tokens for Fasttext transform 160522


/home/ec2-user/SageMaker/ML_hackathon_2019/lib.py:230: RuntimeWarning: invalid value encountered in true_divide
  token2vec = {k: self.model.wv[k] if k in self.model.wv else empty for k in uniq_tokens}


Fasttext Transforms done at: 2019-06-12 20:49:44.720385
Fasttext Transforms start at: 2019-06-12 20:49:44.740831
Number of Unique Test Tokens for Fasttext transform 332242


Fasttext Transforms done at: 2019-06-12 20:50:35.406689


KeyboardInterrupt: 

In [ ]:
start = time()
fasttext = api.load("fasttext-wiki-news-subwords-300") 
print("total = ",(time()-start))

ptr = PreTrainedEmbeddingsTransformer(fasttext,size=300)
ptr.fit()


import gc
gc.collect()

In [ ]:

df_test['fasttext_encoded'] = ptr.transform(df_test['text'].values)
df_test[["ID","GL","text","text_encoded","char","char_encoded","fasttext_encoded"]].to_csv("price_prediction/test.csv",index=False)
gc.collect()

In [ ]:
# Intermediate save
df_train['fasttext_encoded'] = ptr.transform(df_train['text'].values)
df_train[["ID","GL","text","text_encoded","char","char_encoded","fasttext_encoded","PRICE"]].to_csv("price_prediction/train.csv",index=False)

